## Kaggle大赛第一步
前段时间学完了Udacity的深度学习基石课程，内容挺好，但是就是缺乏实战，于是找到了fast.ai的课程，准备进入kaggle大赛来检验下自己的学习成果，这是kaggle大赛第一步：入门。

下面是一些TODO LIST

1. 注册Kaggle的账号
2. 选择[dogs-vs-cats-redux-kernels-edition](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)项目，选择同意数据协议（为了后面通过命令行下载数据用）
3. [aws环境搭建](http://wiki.fast.ai/index.php/AWS_install)
4. 数据目录准备

## download data
第一步，我们要去下载数据，地址：链接: https://pan.baidu.com/s/1kUAnr3T 密码: pdxw

In [1]:
# frequent file shortcuts
import os, sys
current_directory = os.getcwd()
LESSON_HOME_DIR = current_directory
DATA_HOME_DIR = os.path.join(current_directory,"../data/redux")
print(DATA_HOME_DIR)
print(LESSON_HOME_DIR)

/home/carnd/fast.ai/lesson1/../data/redux
/home/carnd/fast.ai/lesson1


In [ ]:
## 创建目录
![ ! -d $DATA_HOME_DIR ] && mkdir -pv $DATA_HOME_DIR

In [ ]:
#Create directories
%cd $DATA_HOME_DIR

In [ ]:
# 命令安装
# 参考文档：https://github.com/floydwch/kaggle-cli
# !pip install kaggle-cli

In [ ]:
# 配置kg
# 参考：http://wiki.fast.ai/index.php/Kaggle_CLI

## Organize data into folders
这一步是为了更好的组织我们的数据，结构如下：
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```

In [ ]:
#Create directories
%cd $DATA_HOME_DIR

In [ ]:
# %mkdir -pv valid # 验证
# %mkdir results # 结果
# %mkdir -pv sample/train # 测试用目录
# %mkdir -pv sample/test
# %mkdir -pv sample/valid
# %mkdir -pv sample/results
# %mkdir -pv test/unknown # 测试

In [ ]:
# !mkdir results
# !mkdir -pv sample/train
# !mkdir -pv sample/test
# !mkdir -pv sample/valid
# !mkdir -pv sample/results
# !mkdir -pv test/unknown # 测试

In [ ]:
%cd $DATA_HOME_DIR/train

In [ ]:
# 随机移动2k张照片到验证目录
import glob
import numpy as np
g = glob.glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], DATA_HOME_DIR+'/valid/' + shuf[i])

In [ ]:
!ls $DATA_HOME_DIR/valid | wc -l

In [ ]:
# copy 200 张到 sample的train目录下
from shutil import copyfile
g = glob.glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], DATA_HOME_DIR+'/sample/train/' + shuf[i])

In [ ]:
!ls $DATA_HOME_DIR/sample/train/ | wc -l

In [ ]:
g = glob.glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], DATA_HOME_DIR+'/sample/valid/' + shuf[i])

## Rearrange image files into their respective directories
重新安排目录

In [ ]:
#Divide cat/dog images into separate directories

%cd $DATA_HOME_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
# Create single 'unknown' class for test set
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/

In [ ]:
%cd $DATA_HOME_DIR
!tree -d

## 建立模型
此处我们没有直接采用fast.ai中的文件，而是决定自己来重新写下，为什么呢？一是为了让自己更好的理解，另一个则是使用Python版本的原因，此处我们使用3.5版本

第一步还是先直接饮用课程中写好的文件，直接运行，看有什么问题的

In [2]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'
utils_path = current_directory + "/../utils"

/home/carnd/fast.ai/data/redux


In [3]:
# !KERAS_BACKEND=theano
from keras import backend
backend.set_image_dim_ordering("th")

Using TensorFlow backend.


In [4]:
backend.backend()

'tensorflow'

In [6]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


In [5]:
#Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(utils_path))
#import modules
from utils import *
# from vgg16 import Vgg16

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

上面遇到的问题，由于是3.5版本，需要修改
```
#import cPickle as pickle
import _pickle as pickle
```
conda install bcolz

## Finetuning and Training

In [ ]:
import importlib
import vgg16
importlib.reload(vgg16)

此处有问题，去看下get_batches里面到底做了什么呢？里面有个重要的函数`ImageDataGenerator`，让我们来看下到底是干什么了

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator( rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

上面其实是通过 ImageDataGenerator 来做了一个数据的增强Augmentation

In [ ]:
![ ! -d $path/preview ] && mkdir $path/preview

In [ ]:
flow = gen.flow_from_directory(train_path,target_size=(224,224),class_mode='categorical',shuffle=False,batch_size=1,
                                  save_to_dir=path+'/preview', save_prefix='dog-cat')

In [ ]:
print(flow.class_indices) # 类别字典
print(len(flow.classes),flow.classes) # 加载进来的标签
print(flow.image_shape) # 图片大小
print(flow.nb_class) # 类别个数
print(flow.nb_sample) # 总个数
print(flow.batch_size)

In [ ]:
i = 0
flow.reset()
for batch in flow:
    i+=1
    # batch is  a tuple (inputs, targets)
    print(type(batch),len(batch),type(batch[0]),batch[0].shape,batch[1].shape,batch[1]) # 具体的图像数据
    if i>8:
        break

In [ ]:
from PIL import Image
from os import listdir
from os.path import isfile, join
prview_path = path+'/preview/'
imagefiles = [ f for f in listdir(prview_path) if isfile(join(prview_path,f)) ]

In [ ]:
imagefiles

In [ ]:
from keras.preprocessing import image

#Helper function to plot images by index in the validation set 
#Plots is a helper function in utils.py
def plots_idx(f, titles=None):
    plots([image.load_img(join(prview_path,i)) for i in f], titles=titles)

In [ ]:
plots_idx(imagefiles[:4])

下一步，我们需要看的函数是 finetune 函数，看看里面到底做了什么

原vgg模型最后一层是一个输出为1000的FC层，此处我们需要将其去掉，然后改为我们需要的2分类器，代码如下：
```
def ft(self, num):
        """
            Replace the last layer of the model with a Dense (fully connected) layer of num neurons.
            Will also lock the weights of all layers except the new layer so that we only learn
            weights for the last layer in subsequent training.

            Args:
                num (int) : Number of neurons in the Dense layer
            Returns:
                None
        """
        model = self.model
        model.pop()
        for layer in model.layers: layer.trainable=False
        model.add(Dense(num, activation='softmax'))
        self.compile()
```

因此 finetune 中做的事情就是重新构建了下模型

接着到了最关键的 fit 方法了，具体训练模型了，方法可以参看[model](https://keras.io/models/model/)，fit 和 fit_generator 的区别是输入是否是 generator

In [ ]:
print(keras.__version__)

In [ ]:
model = Sequential()

In [ ]:
#Signature: model.fit_generator(generator, samples_per_epoch, nb_epoch, 
#   verbose=1, callbacks=None, validation_data=None, nb_val_samples=None, class_weight=None,
#   max_q_size=10, nb_worker=1, pickle_safe=False, initial_epoch=0, **kwargs)
# model.fit_generator()
# 此处使用的 keras 的版本比较老，所有用的是老版本的接口

In [7]:
#import Vgg16 helper class
vgg = Vgg16()

In [8]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=256
no_of_epochs=1

In [9]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
vgg.finetune(batches)

#Not sure if we set this for all fits
vgg.model.optimizer.lr = 0.1

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [10]:
#Notice we are passing in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print("Running epoch: %d" %(epoch))
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print("Completed %s fit operations"%(no_of_epochs))

Running epoch: 0
Epoch 1/1
22912/23000 [============================>.] - ETA: 2s - loss: 2.3178 - acc: 0.8500   

ResourceExhaustedError: OOM when allocating tensor with shape[256,64,224,224]
	 [[Node: Conv2D_1 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](transpose_3, transpose_4)]]
	 [[Node: Mean_3/_185 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_270_Mean_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Conv2D_1', defined at:
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ca

## Generate Predictions

In [ ]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

In [ ]:
#For every image, vgg.test() generates two probabilities 
#based on how we've ordered the cats/dogs directories.
#It looks like column one is cats and column two is dogs
print(preds[:5])

filenames = batches.filenames
print(filenames[:5])

In [ ]:
#You can verify the column ordering by viewing some images
from PIL import Image
Image.open(test_path + filenames[2])

In [ ]:
#Save our test results arrays so we can use them again later
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)

## Validate Predictions
模型验证，我们主要看下面5类数据：

1. A few correct labels at random
2. A few incorrect labels at random
3. The most correct labels of each class (ie those with highest probability that are correct)
4. The most incorrect labels of each class (ie those with highest probability that are incorrect)
5. The most uncertain labels (ie those with probability closest to 0.5).

## 问题记录

1. 在aws环境搭建中，第一个问题是只支持3个地域，离中国太远了。慢，于是想个办法能不能用国内的？所以我换了Udacity-dl的ami，目前只能手动建立，做不到fast.ai中的命令行，暂时没有时间去研究那个脚本

2. 在mac上显示清晰问题
```
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
```